In [1]:
import ray

# Load data.
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column.
test_dataset = valid_dataset.drop_columns(cols=["target"])

2023-06-15 21:27:40,435	WARNING services.py:404 -- Found multiple active Ray instances: {'10.244.0.5:6380', 'service-ray-cluster:6380', '192.168.3.51:6378'}. Connecting to latest cluster at 192.168.3.51:6378. You can override this by setting the `--address` flag or `RAY_ADDRESS` environment variable.
2023-06-15 21:27:40,436	INFO worker.py:1432 -- Connecting to existing Ray cluster at address: 192.168.3.51:6378...
2023-06-15 21:27:40,449	INFO worker.py:1616 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
2023-06-15 21:27:51,406	WARNING read_api.py:358 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.


Read progress 0:   0%|          | 0/1 [00:00<?, ?it/s]

Read progress 0:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
from ray.air.config import ScalingConfig

scaling_config = ScalingConfig(
    # Number of workers to use for data parallelism.
    num_workers=1,
    # Whether to use GPU acceleration.
    use_gpu=True,
)

In [9]:
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config=scaling_config,
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "tree_method": "gpu_hist",  # uncomment this to use GPU for training
        "eval_metric": ["logloss", "error"],
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
)

In [10]:
result = trainer.fit()
print(result.metrics)

(XGBoostTrainer pid=730760) /home/modongsong/.pyenv/versions/miniconda3-4.7.12/envs/jupyterlab/lib/python3.10/site-packages/xgboost_ray/main.py:512: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(XGBoostTrainer pid=730760)   warnings.warn(
(_RemoteRayXGBoostActor pid=730018)   warnings.warn(
(_RemoteRayXGBoostActor pid=730830) [21:32:45] task [xgboost.ray]:140180960923936 got new rank 0
(XGBoostTrainer pid=730760) [21:32:47] WARNING: ../src/gbm/gbtree.cc:415: 
(XGBoostTrainer pid=730760)   Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
(XGBoostTrainer pid=730760)   machine. Consider using `save_model/load_model` instead. See:
(XGBoostTrainer pid=730760) 
(XGBoostTrainer pid=730760)     https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html
(XGBoostTrainer pid=730760) 
(XGBoostTrainer pid=730760)   for more details about differences between saving model and serializing.  Changing `tree_me

Trial name,date,done,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,train-error,train-logloss,training_iteration,trial_id,valid-error,valid-logloss
XGBoostTrainer_19cff_00000,2023-06-15_21-32-49,True,0,mds-hp,21,192.168.3.51,730760,True,6.04572,0.261944,6.04572,1686835969,0,0.0185789,21,19cff_00000,0.0350877,0.0873526


(XGBoostTrainer pid=730760) [21:32:48] WARNING: ../src/gbm/gbtree.cc:415: 
(XGBoostTrainer pid=730760)   Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
(XGBoostTrainer pid=730760)   machine. Consider using `save_model/load_model` instead. See:
(XGBoostTrainer pid=730760) 
(XGBoostTrainer pid=730760)     https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html
(XGBoostTrainer pid=730760) 
(XGBoostTrainer pid=730760)   for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
(XGBoostTrainer pid=730760) [21:32:48] WARNING: ../src/gbm/gbtree.cc:425: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
(XGBoostTrainer pid=730760) [21:32:48] WARNING: ../src/learner.cc:339: No visible GPU is found, setting `gpu_id` to -1
2023-06-15 21:32:49,032	INFO tune.py:945 -- Total run time: 8.35 seconds (8.34 seconds for the tuning loop).


{'train-logloss': 0.01857892927496071, 'train-error': 0.0, 'valid-logloss': 0.08735255994942932, 'valid-error': 0.03508771929824561, 'time_this_iter_s': 0.261944055557251, 'should_checkpoint': True, 'done': True, 'training_iteration': 21, 'trial_id': '19cff_00000', 'date': '2023-06-15_21-32-49', 'timestamp': 1686835969, 'time_total_s': 6.045717239379883, 'pid': 730760, 'hostname': 'mds-hp', 'node_ip': '192.168.3.51', 'config': {}, 'time_since_restore': 6.045717239379883, 'iterations_since_restore': 21, 'experiment_tag': '0'}


2023-06-15 21:52:53,221	WARNING worker.py:1986 -- The autoscaler failed with the following error:
Terminated with signal 15
  File "/home/modongsong/.pyenv/versions/miniconda3-4.7.12/envs/jupyterlab/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 674, in <module>
    monitor.run()
  File "/home/modongsong/.pyenv/versions/miniconda3-4.7.12/envs/jupyterlab/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 550, in run
    self._run()
  File "/home/modongsong/.pyenv/versions/miniconda3-4.7.12/envs/jupyterlab/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 454, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)

[2023-06-15 21:52:53,388 E 729506 729609] core_worker.cc:569: :info_message: Attempting to recover 17 lost objects by resubmitting their tasks. To disable object reconstruction, set @ray.remote(max_retries=0).
